In [5]:
from pathlib import Path
import os
import warnings
import random

import re
import json
import pandas as pd
from datasets import ClassLabel, Dataset, DatasetDict
import librosa
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from IPython.display import Audio

from IPython.display import display, HTML


import torch
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2Processor

In [6]:
with open("/kaggle/input/speech-to-text/train/174/168635/174-168635.trans.txt") as file_name:
    data = file_name.read().split("\n")

In [7]:
file_name=[]
sentences=[]

In [8]:
for i in data:
    file_name.append(str(i[:15]))
    sentences.append(i[15:])

In [9]:
audio_data = {
    'ID':file_name,
    'transcription':sentences
}

In [10]:
data_train = pd.DataFrame(audio_data)


In [11]:
# Create dict of ID's and file paths
paths = {}
path = '/kaggle/input/speech-to-text/train/174/168635'
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".flac"):
            audio_path = os.path.join(root, file)
            p = Path(audio_path)
            id = p.parts[-1].split('.')[0]
            paths[id] = audio_path

In [12]:
Audio(paths[data_train.iloc[2]["ID"].strip()])

In [13]:
data_train['file'] = [paths.get(id) for id in list(data_train['ID'])]

In [14]:
data_train = data_train.drop(["ID"], axis = 1)

In [15]:
data_train

,transcription,file
0,HE HAD NEVER BEEN FATHER LOVER HUSBAND FRIEND,/kaggle/input/speech-to-text/train/174/168635/...
1,THE HEART OF THAT EX CONVICT WAS FULL OF VIRG...,/kaggle/input/speech-to-text/train/174/168635/...
2,HIS SISTER AND HIS SISTER'S CHILDREN HAD LEFT...,/kaggle/input/speech-to-text/train/174/168635/...
3,HE SUFFERED ALL THE PANGS OF A MOTHER AND HE ...,/kaggle/input/speech-to-text/train/174/168635/...
4,ONLY AS HE WAS FIVE AND FIFTY AND COSETTE EIG...,/kaggle/input/speech-to-text/train/174/168635/...
5,COSETTE ON HER SIDE HAD ALSO UNKNOWN TO HERSE...,/kaggle/input/speech-to-text/train/174/168635/...
6,SHE FELT THAT WHICH SHE HAD NEVER FELT BEFORE...,/kaggle/input/speech-to-text/train/174/168635/...
7,THE MAN NO LONGER PRODUCED ON HER THE EFFECT ...,/kaggle/input/speech-to-text/train/174/168635/...
8,NATURE A DIFFERENCE OF FIFTY YEARS HAD SET A ...,/kaggle/input/speech-to-text/train/174/168635/...
9,TO MEET WAS TO FIND EACH OTHER,/kaggle/input/speech-to-text/train/174/168635/...


In [16]:
data_train = data_train.drop(23)

In [17]:
data_train

,transcription,file
0,HE HAD NEVER BEEN FATHER LOVER HUSBAND FRIEND,/kaggle/input/speech-to-text/train/174/168635/...
1,THE HEART OF THAT EX CONVICT WAS FULL OF VIRG...,/kaggle/input/speech-to-text/train/174/168635/...
2,HIS SISTER AND HIS SISTER'S CHILDREN HAD LEFT...,/kaggle/input/speech-to-text/train/174/168635/...
3,HE SUFFERED ALL THE PANGS OF A MOTHER AND HE ...,/kaggle/input/speech-to-text/train/174/168635/...
4,ONLY AS HE WAS FIVE AND FIFTY AND COSETTE EIG...,/kaggle/input/speech-to-text/train/174/168635/...
5,COSETTE ON HER SIDE HAD ALSO UNKNOWN TO HERSE...,/kaggle/input/speech-to-text/train/174/168635/...
6,SHE FELT THAT WHICH SHE HAD NEVER FELT BEFORE...,/kaggle/input/speech-to-text/train/174/168635/...
7,THE MAN NO LONGER PRODUCED ON HER THE EFFECT ...,/kaggle/input/speech-to-text/train/174/168635/...
8,NATURE A DIFFERENCE OF FIFTY YEARS HAD SET A ...,/kaggle/input/speech-to-text/train/174/168635/...
9,TO MEET WAS TO FIND EACH OTHER,/kaggle/input/speech-to-text/train/174/168635/...


In [18]:
import librosa

In [19]:
array, sr = librosa.load(data_train.iloc[2].file)

In [20]:
array.size

349713

In [21]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [22]:
data = DatasetDict({'train': Dataset.from_pandas(data_train[0:16]), 'test': Dataset.from_pandas(data_train[16:])})

In [23]:
data

DatasetDict({
    train: Dataset({
        features: ['transcription', 'file'],
        num_rows: 16
    })
    test: Dataset({
        features: ['transcription', 'file'],
        num_rows: 7
    })
})

In [24]:
chars_to_ignore_regex = '[\"\?\.\!\-\;\:\(\)\,]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

In [25]:
data = data.map(remove_special_characters)

  0%|          | 0/16 [00:00<?, ?ex/s]

  0%|          | 0/7 [00:00<?, ?ex/s]

In [26]:
data

DatasetDict({
    train: Dataset({
        features: ['transcription', 'file'],
        num_rows: 16
    })
    test: Dataset({
        features: ['transcription', 'file'],
        num_rows: 7
    })
})

In [27]:
show_random_elements(data['train'], num_examples=10)

,transcription,file
0,cosette was no longer in rags she was in mourning,/kaggle/input/speech-to-text/train/174/168635/174-168635-0014.flac
1,week followed week these two beings led a happy life in that hovel,/kaggle/input/speech-to-text/train/174/168635/174-168635-0013.flac
2,to meet was to find each other,/kaggle/input/speech-to-text/train/174/168635/174-168635-0009.flac
3,she felt that which she had never felt before a sensation of expansion,/kaggle/input/speech-to-text/train/174/168635/174-168635-0006.flac
4,his sister and his sister's children had left him only a vague and far off memory which had finally almost completely vanished he had made every effort to find them and not having been able to find them he had forgotten them,/kaggle/input/speech-to-text/train/174/168635/174-168635-0002.flac
5,moreover jean valjean had chosen his refuge well,/kaggle/input/speech-to-text/train/174/168635/174-168635-0011.flac
6,he had paid her six months in advance and had commissioned the old woman to furnish the chamber and dressing room as we have seen,/kaggle/input/speech-to-text/train/174/168635/174-168635-0012.flac
7,he had never been father lover husband friend,/kaggle/input/speech-to-text/train/174/168635/174-168635-0000.flac
8,the heart of that ex convict was full of virginity,/kaggle/input/speech-to-text/train/174/168635/174-168635-0001.flac
9,nature a difference of fifty years had set a profound gulf between jean valjean and cosette destiny filled in this gulf,/kaggle/input/speech-to-text/train/174/168635/174-168635-0008.flac


In [28]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [29]:
vocabs = data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
vocab_list = list(set(vocabs["train"]["vocab"][0]))

In [33]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 0,
 'm': 1,
 'd': 2,
 'e': 3,
 'j': 4,
 'o': 5,
 'b': 6,
 'k': 7,
 'n': 8,
 't': 9,
 'y': 10,
 'z': 11,
 'f': 12,
 'u': 13,
 'a': 14,
 's': 15,
 'w': 16,
 'c': 17,
 'l': 18,
 'i': 19,
 'v': 20,
 'h': 21,
 'x': 22,
 "'": 23,
 'r': 24,
 'p': 25,
 'g': 26}

In [34]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [35]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

29

In [36]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [37]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [38]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [39]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [41]:
#import soundfile as sf
import librosa
import warnings

warnings.filterwarnings("ignore")

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch.get('file'), sr = 16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcription"]
    return batch

In [42]:
data = data.map(speech_file_to_array_fn, remove_columns=data.column_names["train"], num_proc=1)

  0%|          | 0/16 [00:00<?, ?ex/s]

  0%|          | 0/7 [00:00<?, ?ex/s]

In [43]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [44]:
data_prepared = data.map(prepare_dataset, remove_columns=data.column_names["train"], batch_size=32, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [45]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [46]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [47]:
!pip install  jiwer

In [48]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [49]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [66]:
processor.tokenizer.vocab_size

29

In [50]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
model.freeze_feature_extractor()

In [52]:
torch.cuda.is_available() 

True

In [53]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./wolof",
  group_by_length=True,
  per_device_train_batch_size=3,
  evaluation_strategy="steps",
  num_train_epochs=12,
  fp16=True,
  save_steps=40000,
  eval_steps=1000,
  logging_steps=1000,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=12,
)

In [55]:
from transformers import Trainer


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_prepared["train"],
    eval_dataset=data_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [56]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


TrainOutput(global_step=72, training_loss=10.364373101128471, metrics={'train_runtime': 130.5295, 'train_samples_per_second': 1.471, 'train_steps_per_second': 0.552, 'total_flos': 6.520990369682187e+16, 'train_loss': 10.364373101128471, 'epoch': 12.0})

In [57]:
model.save_pretrained('wav2vec2-xls-r-300m-english')
processor.save_pretrained('wav2vec2-xls-r-300m-english')

In [62]:
!huggingface-cli login --token


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [68]:
model.push_to_hub("ulrichING/speech_to_text_wave2vect2_english")
processor.push_to_hub('ulrichING/speech_to_text_wave2vect2_english')

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ulrichING/speech_to_text_wave2vect2_english/commit/aef4ce18dca7aa2159f28f311e82eed8d2da3919', commit_message='Upload processor', commit_description='', oid='aef4ce18dca7aa2159f28f311e82eed8d2da3919', pr_url=None, pr_revision=None, pr_num=None)